In [1]:
import numpy as np
import random
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler


#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [2]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [3]:
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [4]:
labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [5]:
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [6]:
filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [7]:
camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [8]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [9]:
#scaler = StandardScaler()
print(np.array(features))
#features = scaler.fit_transform(features)
X = np.array(features)
print(X)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)

[[0.13791114 1.12505555 0.05404324 ... 0.10747871 0.04081609 0.68009406]
 [0.02507781 0.9057585  0.00346441 ... 0.16763815 0.14764351 0.2700713 ]
 [0.09923808 1.09122825 0.01889733 ... 0.07981343 0.04958951 0.37923682]
 ...
 [0.50913167 2.1067946  1.12224829 ... 0.78767842 1.68007588 0.00259321]
 [0.44684452 1.87411916 1.51910186 ... 1.02090526 1.58616257 0.10876646]
 [0.46024311 2.31024432 1.20531154 ... 0.62753201 1.22394812 0.        ]]
[[0.13791114 1.12505555 0.05404324 ... 0.10747871 0.04081609 0.68009406]
 [0.02507781 0.9057585  0.00346441 ... 0.16763815 0.14764351 0.2700713 ]
 [0.09923808 1.09122825 0.01889733 ... 0.07981343 0.04958951 0.37923682]
 ...
 [0.50913167 2.1067946  1.12224829 ... 0.78767842 1.68007588 0.00259321]
 [0.44684452 1.87411916 1.51910186 ... 1.02090526 1.58616257 0.10876646]
 [0.46024311 2.31024432 1.20531154 ... 0.62753201 1.22394812 0.        ]]


In [10]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]

In [11]:
len(X_train)

7368

In [12]:
len(set(y_train))

767

In [13]:
len(X_query)

1400

In [14]:
len(X_gallery)

5328

In [15]:
len(y_train)

7368

In [16]:
X_val = []
y_val = []
camId_val = []
val_ind = []
for i in range(7368):
        if(i not in val_ind):
            X_val.append(X_train[i])
            y_val.append(y_train[i])
            camId_val.append(camId_train[i])
            val_ind.append(i)
            for j in range(7368):
                if(y_train[i] == y_train[j] and i != j):
                    X_val.append(X_train[j])
                    y_val.append(y_train[j])
                    camId_val.append(camId_train[j])
                    val_ind.append(j)
            if ((len(set(y_val)) > 99)):
                break

In [17]:
len((y_val))

966

In [18]:
len(X_val)

966

In [19]:
len(camId_val)

966

In [20]:
len(set(val_ind))

966

In [21]:
X_train_new = []
y_train_new = []
camId_train_new = []
for i in range(7368):
    if(i not in val_ind):
        X_train_new.append(X_train[i])
        y_train_new.append(y_train[i])
        camId_train_new.append(camId_train[i])

In [22]:
len(X_train_new)

6402

In [23]:
len(y_train_new)

6402

In [24]:
len(camId_train_new)

6402

In [25]:
import pandas as pd 
import numpy as np

from keras import layers, optimizers, regularizers
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.models import Sequential

from keras.utils import plot_model
#from kt_utils import *
import keras.backend as K

from sklearn import preprocessing, model_selection 
from keras.wrappers.scikit_learn import KerasRegressor

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [26]:
# create model
model = Sequential()
# layer 1
model.add(Dense(6144, input_dim=6144, activation='relu', kernel_initializer='normal'))
#layer 2
model.add(Dense(1024, activation='relu', kernel_initializer='normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#layer 3
model.add(Dense(128, activation='relu',kernel_initializer='normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#layer 4
model.add(Dense(2, activation='softmax'))
# Compile model
from keras import metrics
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer, loss='categorical_crossentropy') 

In [27]:
X_train_pairs = []
y_train_pair_lables = []
for Xtnn, ytnn in zip(X_train_new, y_train_new):
    randindex = random.randint(0, 6401)
    Xtnn2 = X_train[randindex]
    ytnn2 = y_train[randindex]
    
    randindex = random.randint(0, 6401)
    Xtnn3 = X_train[randindex]
    ytnn3 = y_train[randindex]                 
    
    if((ytnn == ytnn2) and (ytnn == ytnn3)):
        y_train_pair_lables.append((0,0))
    if((ytnn == ytnn2) and (ytnn != ytnn3)):
        y_train_pair_lables.append((0,1))
    if((ytnn != ytnn2) and (ytnn == ytnn3)):
        y_train_pair_lables.append((1,0))
    if((ytnn != ytnn2) and (ytnn != ytnn3)):
        y_train_pair_lables.append((1,1))
    
    Xconcat = np.concatenate((Xtnn,Xtnn2,Xtnn3), axis = None)
    X_train_pairs.append(Xconcat)

y_train_pair_lables = np.array(y_train_pair_lables)
X_train_pairs = np.array(X_train_pairs)
model.fit(X_train_pairs, y_train_pair_lables, batch_size = 80, epochs=20)

Epoch 1/20
6402/6402 [==============================] - 55s 9ms/step - loss: 2.0674
Epoch 2/20
6402/6402 [==============================] - 44s 7ms/step - loss: 1.6067
Epoch 3/20
6402/6402 [==============================] - 44s 7ms/step - loss: 1.4528
Epoch 4/20
6402/6402 [==============================] - 44s 7ms/step - loss: 1.4029
Epoch 5/20
6402/6402 [==============================] - 44s 7ms/step - loss: 1.3887
Epoch 6/20
6402/6402 [==============================] - 44s 7ms/step - loss: 1.3855
Epoch 7/20
6402/6402 [==============================] - 44s 7ms/step - loss: 1.3849
Epoch 8/20
6402/6402 [==============================] - 44s 7ms/step - loss: 1.3848
Epoch 9/20
6402/6402 [==============================] - 44s 7ms/step - loss: 1.3848
Epoch 10/20
6402/6402 [==============================] - 44s 7ms/step - loss: 1.3848
Epoch 11/20
6402/6402 [==============================] - 45s 7ms/step - loss: 1.3848
Epoch 12/20
6402/6402 [==============================] - 45s 7ms/step - lo

In [28]:
def get_acc_score(y_valid, y_q, tot_label_occur):
    recall = 0
    true_positives = 0
    
    k = 0
    
    max_rank = 30
    
    rank_A = np.zeros(max_rank)
    AP_arr = np.zeros(11)
    
    while (recall < 1) or (k < max_rank):
        if (y_valid[k] == y_q):
            
            true_positives = true_positives + 1
            recall = true_positives/tot_label_occur
            precision = true_positives/(k+1)
            
            AP_arr[round((recall-0.05)*10)] = precision
            
            for n in range (k, max_rank):
                rank_A[n] = 1
            
        k = k+1
        
    max_precision = 0
    for i in range(10, -1, -1):
        max_precision = max(max_precision, AP_arr[i])
        AP_arr[i] = max_precision
    
    AP_ = AP_arr.sum()/11
    
    return AP_, rank_A

In [30]:
for val, camId_v, y_v  in zip(X_val, camId_val, y_val):
    v_v_dists = []
    y_valid = []
    for val2, camId_v2, y_v2  in zip(X_val, camId_val, y_val):
        if ((camId_v == camId_v2) and (y_v == y_v2)):
            continue
        else:
            stay = True
            while (stay):
                randindex = random.randint(0, 965)
                if((y_val[randindex] != y_v) and (y_val[randindex] != y_v2)):
                    val3 = X_val[randindex]
                    y_v3 = y_val[randindex]
                    stay = False
            dist = model.predict(np.concatenate((val, val2, val3)).reshape((1,6144)))[0][0]
            print(dist)
            v_v_dists.append(dist)
            y_valid.append(y_v3)

    tot_label_occur = y_valid.count(y_v)
    v_v_dists = np.array(v_v_dists)
    y_valid = np.array(y_valid)

    
    _indexes = np.argsort(v_v_dists)

    # Sorted distances and labels
    v_v_dists, y_valid = v_v_dists[_indexes], y_valid[_indexes]
    print(v_v_dists)

    AP_, rank_A = get_acc_score(y_valid, y_v, tot_label_occur)

    AP.append(AP_)

    rank_accuracies.append(rank_A) 

    #if q  > 5:
    #    break
    #q = q+1

rank_accuracies = np.array(rank_accuracies)

total = rank_accuracies.shape[0]
rank_accuracies = rank_accuracies.sum(axis = 0)
rank_accuracies = np.divide(rank_accuracies, total)

i = 0
print ('Accuracies by Rank:')
while i < rank_accuracies.shape[0]:
    print('Rank ', i+1, ' = %.2f%%' % (rank_accuracies[i] * 100), '\t',
          'Rank ', i+2, ' = %.2f%%' % (rank_accuracies[i+1] * 100), '\t',
          'Rank ', i+3, ' = %.2f%%' % (rank_accuracies[i+2] * 100), '\t',
          'Rank ', i+4, ' = %.2f%%' % (rank_accuracies[i+3] * 100), '\t',
          'Rank ', i+5, ' = %.2f%%' % (rank_accuracies[i+4] * 100))
    i = i+5

AP = np.array(AP)

mAP = AP.sum()/AP.shape[0]
print('mAP = %.2f%%' % (mAP * 100))

0.49918127
0.49957326
0.5004536
0.49922138
0.4998859
0.49856836
0.49893686
0.49894884
0.49846354
0.4987743
0.49796966
0.49981484
0.4994514
0.4990767
0.4985363
0.49959022
0.49910438
0.5003375
0.4992975
0.50031084
0.49992296
0.49945357
0.49938747
0.5004347
0.5003014
0.5014467
0.49984035
0.50015026
0.50113404
0.49989155
0.50064677
0.50099456
0.500833
0.50060064
0.49870646
0.499084
0.5006839
0.49882784
0.4989555
0.49975643
0.49913037
0.49902642
0.4987
0.49968123
0.49938253
0.4994635
0.49981728
0.50019
0.5007972
0.4988809
0.49960738
0.49906567
0.49994755
0.49962124
0.5012182
0.50039107
0.5010477
0.50012565
0.50053835
0.5004243
0.4997932
0.5002656
0.5011556
0.4990385
0.49801382
0.4998822
0.49948218
0.49920556
0.4999834
0.5002406
0.49966133
0.49884707
0.49995813
0.4998128
0.5004763
0.50086766
0.50020427
0.49997073
0.50000435
0.5002283
0.49989033
0.4992723
0.49955782
0.4980214
0.49915996
0.49917033
0.4992385
0.498603
0.49842182
0.49800673
0.49907085
0.49849582
0.49994954
0.49920338
0.49914172


0.49750987
0.49802446
0.4985005
0.49714273
0.49876335
0.4982849
0.49872956
0.49787012
0.49754217
0.49923688
0.49838954
0.50027484
0.49976602
0.5009574
0.50017893
0.4995782
0.49988753
0.49907893
0.5004311
0.5000751
0.50140435
0.5001427
0.5008314
0.5012388
0.49993274
0.5000555
0.5003071
0.50155854
0.5025164
0.49852237
0.4993538
0.49997196
0.49906948
0.49942747
0.5000798
0.50055623
0.5001452
0.49946016
0.4991488
0.50028694
0.5005785
0.50104
0.50044125
0.50109476
0.5014348
0.5021632
0.5011114
0.5004578
0.49967533
0.498451
0.4994832
0.49914145
0.49948868
0.49839458
0.4993235
0.5009241
0.50090575
0.50138605
0.50024486
0.50058144
0.5009487
0.5010979
0.50040007
0.49933442
0.50029916
0.49901876
0.49883562
0.49954614
0.5005991
0.5007906
0.5006588
0.50061977
0.4994462
0.49855468
0.4999425
0.5003311
0.49926946
0.50119
0.50034475
0.50030357
0.50082517
0.49991018
0.50100946
0.50045425
0.500196
0.49967694
0.5011395
0.5009017
0.4999601
0.49958155
0.5002528
0.49933773
0.50068104
0.49969256
0.5001475
0.

IndexError: index 961 is out of bounds for axis 0 with size 961